This is my first attempt to solve the Spaceship Titanic Data science competition on the data science platform Kaggle.

# Preliminary: imports and load the data

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import os

In [ ]:
# variable to determine whether the notebook is running on Collab or not
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

: 

Please make sure to download the data and have it in the same directory as this notebook when running it.

In [ ]:
import os
from zipfile import ZipFile

DOWNLOAD_DIR = "sample_data" if IN_COLAB else os.getcwd() 
file_name = "spaceship-titanic.zip"
file_name_no_zip = "spaceship-titanic"
loc = os.path.join(DOWNLOAD_DIR, file_name)
# opening the zip file in READ mode

if not os.path.isdir(os.path.join(DOWNLOAD_DIR, file_name_no_zip)):
    with ZipFile(loc, 'r') as zip_ref:
        zip_ref.extractall()
        print('Done!')

: 

In [ ]:
train_loc = os.path.join(os.getcwd(), "train.csv")
test_loc = os.path.join(os.getcwd(), "test.csv")

df = pd.read_csv(train_loc)
df_test = pd.read_csv(test_loc)

: 

# Helper functions
in this subsection I write a number of functions to frequently use later.

In [ ]:
# a function to delete columnsfrom both the train and test datasets
def drop_cols(cols, df_train: pd.DataFrame, df_test: pd.DataFrame):
    if isinstance(cols, str):
        df1 = df_train.drop(cols, axis=1)
        df2 = df_test.drop(cols, axis=1)
    else:
        df1 = df_train.drop(columns=cols)
        df2 = df_test.drop(columns=cols)
    return df1, df2

: 

In [ ]:
def new_col_names(old_new_names, df_train ,df_test):
    try:    
        df = df_train.rename(columns=old_new_names)
        df_test = df_test.rename(columns=old_new_names)
        return df, df_test
    except:
        df_no_col = [col for col in old_new_names.keys if col not in df.columns]
        df_test_no_col = [col for col in old_new_names.keys if col not in df_test.columns]
        print("{cols} are not in the {dataf}".format(df_no_col, "training dataset"))
        print("{cols} are not in the {dataf}".format(df_test_no_col, "test dataset"))


: 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def one_hot_encoder(df: pd.DataFrame, features_names: list=None):
    # select all categorical features if none of them were specified
    if features_names is None:
        features_names = df.select_dtypes(include=['object']).columns.tolist()
    # define the encoder
    
    ohe = OneHotEncoder(drop='first', handle_unknown='error', sparse_output=False, dtype=np.int64)
    for f in features_names:        
        try:
            unique_values = np.unique(df[f])
        except:
            unique_values = list(df[f].value_counts().index)
        
        # if there is only one unique value in the column, then this column is to be dropped
        if len(unique_values) <= 1 :
            break
        # the names for the  newly-created features
        new_names = [f'{f}_{u}' for u in unique_values[1:]]
        new_values = ohe.fit_transform(df[[f]])
        print(ohe.categories_)
        df[new_names] = new_values
    
    return df


: 

In [ ]:
! pip install empiricaldist

: 

In [ ]:
from empiricaldist import Cdf

def display_cdf(data: pd.DataFrame, num_feat: str, cat_feat: str, feat_values: list=None):
    if feat_values is None:
        # extract the unique values in the passed feature
        feat_values = list(data[cat_feat].value_counts().index)
    
    # iterate through the different values of feat_values 
    for v in feat_values:
        d = data[data[cat_feat] == v][num_feat]
        data_cdf = Cdf.from_seq(d)
        data_cdf.plot(label=v)

    plt.legend()
    plt.xlabel(f'{num_feat}')
    plt.ylabel(f'cdf of {num_feat}')
    plt.show()   
                

: 

In [ ]:
# Create table for missing data analysis
def draw_missing_data_table(df):
    # find the number of nans in each column
    total = df.isnull().sum().sort_values(ascending=False)
    # calculate the percentage of missing values in each column
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    # convert the results into a dataframe
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

: 

In [ ]:
# let' define a function that applies a function to either 
# the whole dataframe or certain columns on the dataframe

def apply_functions(df_train, df_val, funcs, col_names=None):
    # either have one function passed that should be applied to the whole dataframe
    # or have an equal number of columns and functions where each funtion will be applied to the corresponding column
    all_data = callable(funcs) and col_names is None
    col_funcs = True
    #  if the funcs argument is indeed a function, then the code below will raise an error 
    try:
        col_funcs = (all([callable(f) for f in funcs]) and len(funcs) == len(col_names))
    except:
        col_funcs = False
    
    assert all_data or col_funcs
        
    if col_names is None: # if the function is to be applied on the whole dataframe
        df = df_train.apply(funcs, axis=1)
        df_test = df_val.apply(funcs, axis=1)
    else:
        df = df_train.copy()
        df_test = df_val.copy()

        for col, f in zip(col_names, funcs):
            df[col] = df[col].apply(f)
            df_test[col] = df_test[col].apply(f)
            
    return df, df_test

: 

In [ ]:
def get_col_types(df: pd.DataFrame):
    object_type = "object"
    cat_type = 'category'
    non_num_cols = list(df.select_dtypes([object_type, cat_type]).columns)
    num_cols = list(df.select_dtypes(np.number).columns)
    return num_cols, non_num_cols


: 

In [ ]:
def visualize_interaction(data:pd.DataFrame, feature, use_cols=None, nunique_as_discrete=20, max_cat_values=25):
    if use_cols is None:
        use_cols = data.columns

    # first determine which columns are numerical and which are categorical
    num_feats, cat_feats = get_col_types(data)
    # convert the data into sets
    num_feats = set(num_feats)
    cat_feats = set(cat_feats)
    # consider the discrete features as categorical
    for n in num_feats:
        if len(data.value_counts(n)) <= nunique_as_discrete:
            # consider the numerical feature as categorical
            cat_feats.add(n)
    # remove every element in num_feats that belongs to cat_feats as well
    num_feats = num_feats.difference(cat_feats)

    # consider the case where the passed feat is indeed categorical:
    if feature in cat_feats:
        # iterate through all the columns in the dataset
        for col in use_cols:
            if col == feature: continue

            if col in num_feats:
                sns.boxplot(data=data, x=col, y=feature, orient='h')
                plt.show()
            # only visualize categorical feature with a reasonable number of possible values
            elif len(data.value_counts(col)) < max_cat_values:
                # the hue parameter should be assigned the feature with the smaller number of possible values
                feats = [col, feature]
                feats = sorted(feats, key=lambda x: len(data.value_counts(x)))
                hue_feat, x_feat = feats

                sns.catplot(kind='count', data=data, x=x_feat, hue=hue_feat)
                plt.xticks(rotation=45)
                plt.show()
    else:
        for col in use_cols:
            if col == feature: continue
            
            if col in num_feats:
                # display the cfds of the numerial feature with different values of the categorical one
                display_cdf(data, num_feat=col, cat_feat=feature)
                
            elif len(data.value_counts(col)) < max_cat_values:                
                sns.boxplot(data=data, x=feature, y=col, orient='h')
                plt.show()
                


: 

In [ ]:
from sklearn.model_selection import learning_curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Validation score")

    plt.legend(loc="best")
    return plt

: 

In [ ]:
# these represent a backup of the original data for later use
train_data = pd.read_csv(train_loc)
test_data = pd.read_csv(test_loc)

: 

In [ ]:
def create_submission(estimator, X_train, y, X_test, estimator_name):
    estimator.fit(X_train, y)
    y_pred = pd.DataFrame(data=estimator.predict(X_test), columns=['Transported'])
    y_pred['PassengerId'] = test_data['PassengerId']
    y_pred['Transported'] = y_pred['Transported'].astype(bool)
    # y_pred = y_pred.loc[:, ['PassengerId', 'Transported']]
    # set the index to the passenger's id
    y_pred.set_index("PassengerId", inplace=True)
    y_pred.to_csv(os.path.join(os.getcwd(), f"{estimator_name} submission.csv")) 
    return y_pred
    

: 

# MvP
In this section, I will try to build a basic, minimal machine learning pipeline to estimate the baseline performance and later proceed to enrich the data.

In [ ]:
df.head()
# let's first seperate the predictors from the label
# rename the label column to 'y'
df = df.rename(columns={"Transported":"y"})
df['y'] = df['y'].astype(int)
y = df['y']

df, df_test = drop_cols("PassengerId", df, df_test)

df, df_test = new_col_names(str.lower, df, df_test)

new_names = {"foodcourt":"food", "shoppingmall": "mall", "roomservice": "room",
            "homeplanet":"home", "destination": "des", "vrdeck":"deck", "cryosleep":"sleep"}

df, df_test = new_col_names(new_names, df, df_test)

df_org = df.copy()
df_test_org = df_test.copy()

: 

In [ ]:
df.info()

: 

In [ ]:
# we can see we have a couple of missing values at each column. Nevertheless with very small portions.
# It might be useful to imput them accordingly.
# let's see how many rows have at least one nan  value 
print(df.shape)
df_non_nan = df.dropna()
df_non_nan.shape
# losing more than 2000 rows is indeed not a good idea. Let's try to impute the values

: 

In [ ]:
num, non_num = get_col_types(df)

: 

In [ ]:
# let's try to understand how the different column behaves in general
for c in non_num:
    if len(df.value_counts(c)) <= 10:
        sns.catplot(kind='count', data=df, x=c, hue='y')
        plt.show()      

: 

Remarks:
* among the categorial features, the most informative feature seems to be CyroSleep. Sleeping passengers are likely to be teleported 
* VIP is not really that informative, as for both VIP and non-VIP, the passengers are almost equally distributed 
* destination and homePlanet as they are currently might not be really informative
* maybe some insight could be extracted from cabin and name: at the moment, they will be dropped


In [ ]:
# let's drop the name and cabin features
df, df_test = drop_cols(['name', 'cabin'], df, df_test)

: 

In [ ]:
# let's consider the distributions of numerical columns
print(df.describe()) 

: 

it is easy to see that the maximum values for each of the services in the Titanic SpaceShip are extravagent to say the least.
This should be investigated further.

In [ ]:
num_cols, non_num_cols = get_col_types(df)
for n in num_cols:
    g = df[n].plot(kind='hist')
    g.set(xlabel=f"values of column {n}", ylabel="Frequency")
    g.set_title(f'The distribution of column {n}')
    plt.show()

: 

In [ ]:
# let's see how these features correlate with "y"
from empiricaldist import Cdf
for n in num_cols[:-1] :
    d1 = df[df['y'] == 0][n]
    d2 = df[df['y'] == 1][n]
    cdf1 = Cdf.from_seq(d1)
    cdf1.plot(label="y = 0")
    
    cdf2 = Cdf.from_seq(d2)
    cdf2.plot(label="y = 1")
    plt.legend()
    plt.xlabel(f'{n}')
    plt.ylabel(f'cdf of {n}')
    plt.show()   

    

: 

* We can see that roomservice + vrdeck + spa share very similar distribution between the different classes (transported and not transported): they can be possibly be combined into a single feature
* the foodcourt and shoppingmall seem to be similar as well


In [ ]:
# let's try to understand the percentage of missing values at each column
draw_missing_data_table(df)

: 

only few missing values at each column, using "mean" or "median" imputing does not seem like a very bad idea for numerical columns
as for cyrosleep and vip, it might be worth veryfing a certain assumption
1. people in cyrosleep do not spend money at all: 
2. vip: spend significantly more than the rest of the passengers

In [ ]:
# first let's add a total_spending column where it calculates the sum of money spent on different services

spending_cols = ['deck','room','food','spa', 'mall']
def total_spent(row):
    spending_cols = ['deck','room','food','spa', 'mall']
    row['total'] = row[spending_cols].sum()
    return row
df, df_test = apply_functions(df, df_test, total_spent)

: 

In [ ]:
# let's verify our hypothesis

sns.boxplot(data=df, x='total', y='sleep', orient='h')
plt.show()

# let's verify the vip hypothesis

data = df[(df['total'] > 0)]
sns.boxplot(data=data, x='total', y='vip', orient='h')
plt.show()

: 

In [ ]:
# the first hypothesis is indeed verified: 
# total == 0 <=> passenger is asleep

# the second hypothesis, is relatively in the correct direction, the mean spending of a vip passenger is significantly 
# larger than the non-vip passenger. it is not possible to determine with high certainty that a high spender is a vip

def impute_total_sleep(row):
    if row['total'] == 0 or row['sleep']:
        for c in spending_cols:
            row[c] = 0
        row['sleep'] = True
        
    return row

: 

In [ ]:
spending_cols.append('total')
df_spent = df[df['total']>0]
df_spent_vip = pd.pivot_table(data=df_spent, index=['vip'], values=spending_cols, aggfunc=['median'])
print(df_spent_vip)

: 

In [ ]:
import math

def impute_total_vip(row):
    if math.isnan(row['vip']):
        # set the rows with 'vip' missing values to either True or False depending on the
        # total spending 
        if row['total'] > df_spent_vip.loc[False, ('median', 'total')]:
            row['vip'] = True
        else:
            row['vip'] = False
     
    for c in spending_cols:
        if math.isnan(row[c]):
            row[c] = df_spent_vip.loc[row['vip'], ('median', c)]
    return row

df, df_test = apply_functions(df, df_test, impute_total_sleep)
df, df_test = apply_functions(df, df_test, impute_total_vip)

: 

In [ ]:
# let's use the median for the moment to impute age
# and the mode for des and home features
from sklearn.impute import SimpleImputer

mode_imputer = SimpleImputer(strategy='most_frequent')
median_imputer = SimpleImputer(strategy='median')

# impute des and home features

planets=['home', 'des']

df [planets] = pd.DataFrame(data=mode_imputer.fit_transform(df[planets]), columns=planets)
df [['age']] = pd.DataFrame(data=median_imputer.fit_transform(df[['age']]), columns=['age'])

df_test[planets] = pd.DataFrame(data=mode_imputer.transform(df_test[planets]), columns=planets)
df_test[['age']] = pd.DataFrame(data=median_imputer.transform(df_test[['age']]), columns=['age'])


: 

In [ ]:
# as expected we have no missing values anymore
draw_missing_data_table(df)

: 

In [ ]:
# final step in the baseline pipeline is to encode the categorial features 
# and convert the boolean ones numerical

df = one_hot_encoder(df, planets)
df_test = one_hot_encoder(df_test, planets)

# remove the original columns
df, df_test = drop_cols(planets, df, df_test)

def set_bool_int(row):
    row['sleep'] = int(row['sleep'])
    row['vip'] = int(row['vip'])
    return row

df, df_test = apply_functions(df, df_test, set_bool_int)


: 

before proceeding with the evaluation, I am using an external python file with a number of helper functions mainly concerning hyperparameter tuning and model selection. The cell below will download it for you!!

In [ ]:
! wget https://raw.githubusercontent.com/ayhem18/My_Kaggle_Repo/space_titanic/Competitions/SpaceshipTitanic/classifiers.py

: 

In [ ]:
# let's remove the 'y' column from the training dataset 
df.pop('y')

: 

In [ ]:
from classifiers import tune_model
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=5000)

grid= {"C":np.logspace(-3, 3, 20)}

lr = tune_model(lr, grid, df, y)


: 

In [ ]:
title = "learning curve: Logistic Regression"
cv = 10
plot_learning_curve(lr, title, df, y, cv=cv,n_jobs=-1)

: 

In [ ]:
# even though it is clear that Logistic Regression is underfitting the problem
# let's test it on the platfor
create_submission(lr, df, y, df_test, "Logistic Regression")
# the baseline submission scored an accuray of 0.78816
# let's see how far accurate our model can be!!

: 

# improving further
In this section I will try to apply a number of feature engineering techniques, further data cleaning and analysis to boost the model's performance. 
Among the points to address:
* attempt to extract insights from cabin and name columns
* address the outliers in the spending columns + vip column
* better encoding and impuation for home and des columns


In [ ]:
# let's reconsider the original dataset
df = df_org.copy()
df_test = df_test_org.copy()
# let's first add the total column
# as well as impute the sleep and the spending columns
df, df_test = apply_functions(df, df_test, total_spent)
df, df_test = apply_functions(df, df_test, impute_total_sleep)

: 

## does destination or origin matter ?

In [ ]:
# let's see how 'home' and 'des' features interact with the rest of the features
# outside of the spending columns, consider only the "total" feature 
use_cols = [c for c in df.columns if c not in ['mall', 'food', 'room','deck', 'spa']]
# let's first consider the 'home' feature
visualize_interaction(df, "home", use_cols=use_cols)

: 

* Earth has the most passengers
* Europa's passengers do not visit PSO
* Earth has no vip people
* Europe is the richest planet and with 2 thirds of its passengers getting transported

In [ ]:
# after calling 
# visualize_interaction(df, 'des', use_cols=use_cols) 
# the 'des' features was deemed non-informative
# thus, it seems reasonable to binarize the 'home' feature and drop the 'des' feature

# let's calculate q3 for Mars
q3_mars = np.quantile(df[df['home'] == 'Mars'], [0.75])


# let's binarize the 'home' feature while 
def new_home(row):
    # let's try to impute the value if it is nan:
    try:
        if math.isnan(row['home']):
            # if the destination is 'POS'... then "home" is not Europe
            try:
                if row['des'].lower().startwith('pos') or row['total'] <= q3_mars:
                    row['is_Europe'] = False
                else: 
                    row['is_Europe'] = True
            except ValueError:
                # this block is accessed if row['des'] or row['total'] is nan
                # statistically, having the passenger's home as Europe is less likely than other planets
                row['is_Europe'] = False   
       
    except TypeError:
        # a TypeError means row['home'] is not Nan
        row['is_Europe'] = row['home'] == 'Europe'

    return row


In [ ]:

df, df_test = apply_functions(df, df_test, new_home)
# drop the home and des features
df, df_test = drop_cols(['des', 'home'], df, df_test)

## Cabin, Passenger_Id, Name: any insights ? 
After reading the data description more carefully, it can be extrapolated that cabin, passenger_id and name column are not completely useless. Let's see if some feature engineering could put these columns to use

In [ ]:
# let's first restore the passenger_id column
df['id'] = train_data['PassengerId']
df_test['id'] = test_data['PassengerId']

def group_number(row):
    g, n = [int(s) for s in row['id'].strip().split("_")]
    row['group'] = g
    row['number'] = n
    return row

df, df_test = apply_functions(df, df_test, group_number)


In [ ]:
def split_cabin(row):
    try:
        d, n, side = row['cabin'].split("/")
        row['deck_num'] = d
        row['cabin_num'] = n
        row['side'] = side
    except:
        pass
    return row

def split_name(row):
    try:
        row['name'] = row['name'].strip().split(" ")[1].lower()
    except:
        pass
    return row

df, df_test = apply_functions(df, df_test, split_cabin)
df, df_test = apply_functions(df, df_test, split_name)   


In [ ]:
new_feats = ['deck_num', 'cabin_num', 'side', 'name', 'group', 'number']
for f in new_feats:
    u = df[f].value_counts()
    if len(u) <= 10:   
        print(df[f].value_counts())
    else:
        print(f"feature {f} has {len(u)} unique values")

We can see that there are many families (more than 2200 unique surname). The feature as it is, is not very informative. However, we can assign a number of family members to each passenger. This value can be approximated with high certainty by the value_counts() function call on the number feature
however it can be done definitely using the name feature.   
The group, number, cabin_num and name (after preprocessing) features will be discarded 


In [ ]:
# let's use the entire dataset for extracting the familities 
alldata = pd.concat([df.drop('y', axis=1), df_test])
group_sizes = alldata.value_counts('group')
fam_sizes = alldata.value_counts('name')

def set_group_size(row):
    try:
        row['group'] = group_sizes.loc[row['group']]
    except:
        pass
    return row

def set_fam_size(row):
    try:
        row['fam_size'] = fam_sizes.loc[row['name']]
    except:
        row['fam_size'] = row['group']
    return row 


# make sure to apply the set_group_size first
df, df_test = apply_functions(df, df_test, set_group_size)
df, df_test = apply_functions(df, df_test, set_fam_size)


In [ ]:
# shit
# let's see how the numerical features varies different families
# survival rate
fam_survival = pd.pivot_table(df, index='name', values=['y'], aggfunc=['mean', 'count'])
# print(fam_survival.sort_values(by=('mean', 'y'), ascending=False)) 
# let's visualize the the distribution of the survival rate among families
fam_survival.reset_index()[('mean', 'y')].plot(kind='hist')
plt.show()

### who survives in a family ?

In [ ]:
# shit\

# let's consider which factors determine the transport of family members
# age mainly ? 
all_data_fam_age = alldata.sort_values(by=['name', 'age'])
fam_age = pd.pivot_table(alldata, index='name', values=['age'], aggfunc='mean')
def set_age_per_fam(row):
    global fam_age
    try:
        row['age_frac'] = row['age'] / fam_age.loc[row['name'], 'age']
    except KeyError:
        pass
    return row
print(fam_age)
df_c, df_t_c = apply_functions(df, df_test, set_age_per_fam)

In [ ]:
df, df_test = apply_functions(df, df_test, set_age_per_fam)

In [ ]:
# let's better understand the distribution of this value
d2 = df[df['y'] == 1]['age_frac']
d1 = df[df['y'] == 0]['age_frac']
cdf1 = Cdf.from_seq(d1)
cdf1.plot(label="y = 0")

cdf2 = Cdf.from_seq(d2)
cdf2.plot(label="y = 1")
plt.legend()
plt.xlabel('age_frac')
plt.ylabel('age_frac')
plt.show()   

In [ ]:
# do not forget to remove the cabin, and idPassenger features
df, df_test = drop_cols(['cabin', 'id'], df, df_test)

# drop the unncessary features
# df, df_test = drop_cols(['name', 'number', 'group', 'cabin_num'], df, df_test)

In [ ]:
# let's consider the distribution of the new features
new_feats = ['deck_num', 'side', 'fam_size']
for nf in new_feats:
    sns.catplot(kind='count', data=df, x=nf, hue='y')
    plt.show()

In [ ]:
# let's consider the spending per family
df['total_per_fam'] = df['total'] / df['fam_size'] 
df_test['total_per_fam'] = df_test['total'] / df_test['fam_size'] 


sns.histplot(data=df, x='total_per_fam')
plt.show()

In [ ]:
# the new features might have certain interactions with both vip and total features
# fam_size
sns.relplot(data=df, x='fam_size', y='total', hue='y', size='vip')
plt.show()
# cabin_num
sns.boxplot(data=df, y='deck_num', x='total', orient='h', hue='y')
plt.show()

## SPENDING OUTLIERS

In [ ]:
# let's consider the non-sleeping customers
# for c in spending_cols[:-1]:
#     g = df[df[c] != 0][c].plot(kind='hist')
#     g.set(xlabel=f"values of column {n}", ylabel="Frequency")
#     g.set_title(f'The distribution of column {n}')
#     plt.show()    


In [ ]:
# the distribution barely change even after excluding the rows with value zero.
# before proceeding with addressing the outliers let's consider the relation between the different spending columns


# spending_cols
# spending_df = df.loc[:, spending_cols].fillna(0).reset_index(drop=True)
# print(spending_df.head())

In [ ]:
# let's consider the those who spend at least some money

# for c1 in spending_cols[:-1]:
#     for c2 in spending_cols[:-1]:
#         if c1 != c2:
#             sns.relplot(data=df[df['total'] != 0], x=c1, y=c2, kind='scatter')
#             plt.show()

The main observation is that under a certain trehshold most pair of spending are highly positevely correlated. Nevertheless, as the spending increases, most pair of spending start correlating negatively.
Let's consider the the IQR values to detect outliers and consider in more depth their behavior

In [ ]:
# let's define a function to extract the necessary percentiles
def outliers (df:pd.DataFrame, spending_col):  
    data = df[df[spending_col] > 0]
    q1, q3 = np.quantile(np.asarray(data[spending_col]), [0.25, 0.75])
    iqr = q3 - q1
    min_ = max(0, q1 - 1.5 * iqr)
    max_ = q3 + 1.5 * iqr
    return min_, max_

spending_limits = dict(zip(spending_cols, [outliers(df, c) for c in spending_cols]))


In [ ]:
spending_limits

In [ ]:
sp_out = 'spending_outlier'
def set_spending_outliers(row):
    global spending_limits
    spending_cols = ['deck','room','food','spa', 'mall', 'total']
    row[sp_out] = 0
    for c in spending_cols:
        limits = spending_limits[c]
        if row[c] > limits[1] or row[c] < limits[0]:
            row[sp_out] += 1
    return row


df, df_test = apply_functions(df, df_test, set_spending_outliers)

In [ ]:
# what is special about these outlier
df_outliers = df[df[sp_out] > 0]
num, cat = get_col_types(df_outliers)

print(df_outliers['y'].value_counts())
print(f"{df_outliers['y'].value_counts()[0] / df_outliers['y'].value_counts().sum()} of the outliers are not transported")

In [ ]:
num, cat = get_col_types(df_outliers)

In [ ]:
for n in num:
    d2 = df_outliers[df_outliers['y'] == 1][n]
    d1 = df_outliers[df_outliers['y'] == 0][n]
    cdf1 = Cdf.from_seq(d1)
    cdf1.plot(label="y = 0")

    cdf2 = Cdf.from_seq(d2)
    cdf2.plot(label="y = 1")
    plt.legend()
    plt.xlabel(f'{n}')
    plt.ylabel(f'cdf of {n}')
    plt.show()   
    # as soon as the spending exceeds a certain threshold, the exact amount spent is no longer significant.
    # let's understand what makes outliers get transported then

In [ ]:
# let's check how it works for total_per_family


# d2 = df_outliers[df_outliers['y'] == 1]["total_per_fam"]
# d1 = df_outliers[df_outliers['y'] == 0]['total_per_fam']
# cdf1 = Cdf.from_seq(d1)
# cdf1.plot(label="y = 0")

# cdf2 = Cdf.from_seq(d2)
# cdf2.plot(label="y = 1")
# plt.legend()
# plt.xlabel("total_per_fam")
# plt.ylabel("total_per_fam")
# plt.show()   


In [ ]:
# let's consider the age 

# sns.boxplot(data=df_outliers, x='age', y='y', orient='h')
# plt.show()

# sns.boxplot(data=df_outliers, x=sp_out, y='y', orient='h')
# plt.show()

# d1 = df_outliers[df_outliers['y'] == 0]['fam_size']
# d2 = df_outliers[df_outliers['y'] == 1]["fam_size"]
# cdf1 = Cdf.from_seq(d1)
# cdf1.plot(label="y = 0")

# cdf2 = Cdf.from_seq(d2)
# cdf2.plot(label="y = 1")
# plt.legend()
# plt.xlabel(f'{n}')
# plt.ylabel(f'cdf of fam_size')
# plt.show() 

In [ ]:
for c in cat:
    if len(df_outliers.value_counts(c)) <= 10:
        sns.catplot(kind='count', data=df_outliers, x=c, hue='y')
        plt.xticks(rotation=45)
        plt.show()      

In [ ]:
# from scipy.stats import boxcox
# for c in spending_cols[:-1]:
#     df[c], lamda = boxcox(1 + df[c])
#     # use the lambda returned by the first call to transform the test dataset
#     df_test[c] = boxcox(1 + df_test[c], lmbda = lamda)

In [ ]:
# for n in num_cols:
#     g = df[n].plot(kind='hist')
#     g.set(xlabel=f"values of column {n}", ylabel="Frequency")
#     g.set_title(f'The distribution of column {n}')
#     plt.show()

In [ ]:
# as these numerical columns are now consideredvery close to normal,applying the z-score strategy seems like a good idea 


# housing['LQFSF_Stats'] = stats.zscore(housing['Low Qual Fin SF'])
# housing[['Low Qual Fin SF','LQFSF_Stats']].describe().round(3)
